In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')

In [2]:
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor
import torch


processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [6]:
from qwenvl.data.input_processor import InputProcessor


ip = InputProcessor(processor)


In [10]:
from qwenvl.argument import DataArguments
from qwenvl.data.openbiomedvid import OpenbiomedvidDataset

OpenbiomedvidDataset('openbiomedvid', ip, ip.proc_args, DataArguments())

2025-07-02 14:49:42,325 - qwenvl.data.base - INFO - Loading dataset withcomment/openbiomedvid from /projects/cft_vlm/datasets/openbiomedvid/data/dataset
2025-07-02 14:49:42,339 - qwenvl.data.sft - INFO - self.need_num_content_tokens()=False


Counting total tokens (num_proc=24):   0%|          | 0/44188 [00:00<?, ? examples/s]

Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processor and will be ignored.
Keyword argument `text_only` is not a valid argument for this processo

Filtering too long items (num_proc=24):   0%|          | 0/44188 [00:00<?, ? examples/s]

2025-07-02 14:50:05,089 - qwenvl.data.sft - INFO - Found 2168 / 44188 (0.0491) items with more than 16384 tokens.
2025-07-02 14:50:05,090 - qwenvl.data.sft - INFO - Dataset withcomment/openbiomedvid has 42020 0.9509 items after filtering.
2025-07-02 14:50:05,105 - qwenvl.data.sft - INFO - Data packing is enabled.
2025-07-02 14:50:05,591 - qwenvl.data.sft - INFO - Loading bins from pickle file /projects/cft_vlm/datasets/openbiomedvid/data/dataset/bins.pkl.
2025-07-02 14:50:05,602 - qwenvl.data.sft - INFO - Packing dataset into 25880 bins.


Keyword argument `text_only` is not a valid argument for this processor and will be ignored.


2025-07-02 14:50:05,607 - qwenvl.data.sft - INFO - Example item: ['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|><|im_end|>\n<|im_start|>user\nWhat specific condition is highlighted in the video related to the position of the patella?\n<|im_end|>\n<|im_start|>assistant\nPatella alta\n<|im_end|>\n<|im_start|>user\nWhat anatomical feature is primarily responsible for the high position of the patella in the video?\n<|im_end|>\n<|im_start|>assistant\nElongated patellar tendon\n<|im_end|>\n<|im_start|>user\nWhat is observed on the inferior articular surface of the patella in the video?\n<|im_end|>\n<|im_start|>assistant\nThinning and minor irregularities\n<|im_end|>\n<|im_start|>user\nWhat type of stress does the patella experience due to patella alta as shown in the video?\n<|im_end|>\n<|im_start|>assistant\nLateral subluxation stress\n<|im_end|>\n']


In [18]:
for k, v in ip.__dict__.items():
  if k not in ip2.__dict__:
    print(f"Missing in ip2: {k}")
  else:
    if v != ip2.__dict__[k]:
      print(f"Different in ip2: {k}")
    else:
      print(f"Same in ip2: {k}")

Same in ip2: image_token
Same in ip2: video_token
Same in ip2: image_token_id
Same in ip2: video_token_id
Same in ip2: vs_id
Same in ip2: ve_id
Same in ip2: sys_prompt
Same in ip2: mode
Same in ip2: add_generation_prompt
Same in ip2: add_vision_id
Same in ip2: ignore_idx
Same in ip2: proc_args
Same in ip2: chat_template
Different in ip2: image_processor
Different in ip2: tokenizer
Different in ip2: video_processor


In [ ]:
ip.save_pretrained("test_processor")

In [3]:
from qwenvl.argument import ProcessingArguments
from qwenvl.data.utils import make_model_input
from pathlib import Path
vid_dir = Path('/projects/cft_vlm/datasets/openbiomedvid/data/vid_processed')

conversations = [[
  {"role": "user",
   "content": [
    {'text': "Describe the video.\n"},
    {'video': vid_dir / '00cD-55fnaw_2590_3016.mp4'}
    ]},
  {"role": "assistant",
   "content": [
    {'text': "This video describes.\n"}
    ]}],
  [{"role": "user",
   "content": [
     {'text': "what is in this video?"},
     {'video': vid_dir / '01KY3GQKBzg_14_158.mp4'}
   ]},
  {"role": "assistant",
   "content": [{'text': "this video shows a person doing something interesting.\n"}]
  }
]]
proc_args = ProcessingArguments()
data_dict, text0 = make_model_input([conversations], processor, proc_args, for_training=True, mode='ift')

ImportError: cannot import name 'make_model_input' from 'qwenvl.data.utils' (/fs01/projects/cft_vlm/finetune/qwenvl/data/utils.py)

In [ ]:
data_di

In [4]:
import torch
import torch.nn.functional as F
from typing import List, Literal


def pad_and_stack_tensors(
    tensors: List[torch.Tensor],
    target_length: int,
    padding_value: int = 0,
    padding_side: Literal["right", "left"] = "right",
    truncation_side: Literal["right", "left"] = "right",
    **kwargs,
) -> torch.Tensor:
  """
  Pads or truncates a list of 1D tensors to a target length and stacks them.

  This function processes a list of 1D tensors to ensure they all have the
  same specified `target_length`. Tensors shorter than the target are padded,
  and tensors longer than the target are truncated. The processed tensors are
  then stacked into a single 2D tensor.

  Args:
      tensors (List[torch.Tensor]):
          A list of 1D PyTorch tensors to process.
      target_length (int):
          The desired final length for each tensor.
      padding_value (int, optional):
          The value to use for padding. Defaults to 0.
      padding_side (Literal["right", "left"], optional):
          The side to add padding to if a tensor is shorter than
          `target_length`. Defaults to "right".
      truncation_side (Literal["right", "left"], optional):
          The side to truncate from if a tensor is longer than
          `target_length`. Defaults to "right".

  Returns:
      torch.Tensor:
          A 2D tensor of shape (len(tensors), target_length) containing the
          processed and stacked tensors. If the input list is empty, an empty
          tensor of shape (0, target_length) is returned.

  Raises:
      ValueError: If `padding_side` or `truncation_side` are not "left" or "right".
  """
  # --- 1. Input Validation ---
  if padding_side not in ["right", "left"]:
    raise ValueError(
        f"padding_side must be 'right' or 'left', but got '{padding_side}'")
  if truncation_side not in ["right", "left"]:
    raise ValueError(
        f"truncation_side must be 'right' or 'left', but got '{truncation_side}'")

  # --- 2. Handle Edge Case: Empty Input List ---
  if not tensors:
    # Return an empty tensor with the correct shape.
    # Assuming torch.long for token IDs, but could be adapted.
    return torch.empty((0, target_length), dtype=torch.long)
  attention_masks = [] # <-- ADDED
  processed_tensors = []
  # Use the device of the first tensor for consistency
  device = tensors[0].device

  # --- 3. Process each tensor ---
  for tensor in tensors:
    current_len = tensor.size(0)

    # Ensure tensor is 1D
    if tensor.dim() != 1:
      raise ValueError(
          f"All tensors in the input list must be 1D, but found a tensor with shape {tensor.shape}")

    if current_len >= target_length:
      if truncation_side == 'right':
        processed_tensor = tensor[:target_length]
      else:
        processed_tensor = tensor[-target_length:]
      mask = torch.ones(target_length, dtype=torch.long, device=device)
    
    elif current_len < target_length:
      pad_len = target_length - current_len
      if padding_side == 'right':
        padding = (0, pad_len)
      else:
        padding = (pad_len, 0)
    
      processed_tensor = F.pad(tensor, padding, "constant", padding_value)
      mask = torch.ones(current_len, dtype=torch.long, device=device)
      mask = F.pad(mask, padding, "constant", 0)
        
    processed_tensors.append(processed_tensor)
    attention_masks.append(mask)
    
  stacked_tensors = torch.stack(processed_tensors)
  stacked_masks = torch.stack(attention_masks)

  return stacked_tensors, stacked_masks # <-- MODIFIED

In [6]:
for k in 'labels', 'input_ids', 'attention_mask':
  print(k)
  print('----', features[k].shape, data_dict[k].shape, torch.equal(features[k], data_dict[k]))

labels
---- torch.Size([1, 26878]) torch.Size([1, 26878]) True
input_ids
---- torch.Size([1, 26878]) torch.Size([1, 26878]) True
attention_mask
---- torch.Size([1, 26878]) torch.Size([1, 26878]) True


In [7]:
for k in 'image_grid_thw', 'video_grid_thw', 'second_per_grid_ts', 'pixel_values_videos', 'pixel_values_images':
  print(k)
  fk = features.get(k, None)
  dk = data_dict.get(k, None)
  if fk is None or dk is None:
    print('    ', fk is not None, dk is not None)
  else:
    print('    ', fk.shape if fk is not None else None, dk.shape if dk is not None else None, torch.equal(fk, dk))

image_grid_thw
     False False
video_grid_thw
     torch.Size([2, 3]) torch.Size([2, 3]) True
second_per_grid_ts
     torch.Size([2]) torch.Size([2]) True
pixel_values_videos
     torch.Size([107280, 1176]) torch.Size([107280, 1176]) True
pixel_values_images
     False False


In [8]:
data_dict.second_per_grid_ts

tensor([2.0262, 2.0000])